<a href="https://colab.research.google.com/github/RuinedPenguin/SMAI-IIITH-Project-Lenet/blob/object-oriented-approach/Lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import cupy as cp
# when gpu

In [1]:
import numpy as cp
# when cpu

In [2]:
from abc import ABC, abstractmethod

In [25]:
# pip install zope
# import zope.interface

In [27]:
# cp.vstack((cp.array([[1,0,0]]).T, 1))

## **Lenet**

In [3]:
class Layer(ABC):

  @abstractmethod
  def initialise_weights(self, input_shape):
    pass

  @abstractmethod
  def forward(self, x):
    pass

  @abstractmethod
  def backward(self):
    pass

In [4]:
class Conv(Layer):
  def __init__(self, k, c_out):
    self.k = k
    self.c_out = c_out
    self.kernels = []
    # self.p = p
    # self.s = s

  def initialise_weights(self, input_shape):
    if len(self.k) == 2:
      h,w = self.k
      c = 0
    else:
      h,w,c = self.k
    
    for i in range(self.c_out):
      if (c == 0):
        self.kernels.append(cp.random.rand(h,w))
      else:
        self.kernels.append(cp.random.rand(h,w,c))

    return (input_shape[0] - self.k[0] + 1, 
            input_shape[1] - self.k[1] + 1, 
            self.c_out)

  def forward(self, x):
    new_h = x.shape[0] - self.k[0] + 1
    new_w = x.shape[1] - self.k[1] + 1

    # if self.c_out == 1:
    #   return convolve(x, self.kernels[0], new_h, new_w)
    
    out = cp.zeros((new_h, new_w, self.c_out))

    for i in range(self.c_out):
      out[:,:,i] = convolve(x, self.kernels[i], new_h, new_w)

    return out

  def convolve(inp, kernel, new_h, new_w):
    out = cp.zeros((new_h, new_w))

    for i in range(new_h):
      for j in range(new_w):
        out[i,j] = inp[i : i +kernel.shape[0], j : j +kernel.shape[0]] * kernel

    return out

  def backward(self):
    pass

In [5]:
class Pool(Layer):
  def __init__(self, k):
    self.k = k
    # self.p = p
    # self.s = s

  def initialise_weights(self, input_shape):
    if len(input_shape) == 2:
      return (input_shape[0] // self.k, input_shape[1] // self.k)
    
    return (input_shape[0] // self.k, input_shape[1] // self.k, 
            input_shape[2])
    

  def forward(self, x):
    if len(x.shape) == 2:
      return self.pool_2d(x)
    
    out = cp.zeros((x.shape[0] // self.k, x.shape[1] // self.k, x.shape[2]))
    for i in range(x.shape[2]):
      out[:,:,i] = self.pool_2d(x[:,:,i])
    
    return out

  def pool_2d(self, x_2d):
    out = cp.zeros((x_2d.shape[0] // self.k, x_2d.shape[1] // self.k))
    for i in range(out.shape[0]):
      for j in range(out.shape[1]):
        h = i * self.k
        w = j * self.k
        out[i,j] = self.pooling_function(x_2d[h:h+self.k, w:w+self.k])

    return out

  @abstractmethod
  def pooling_function(self, matrix):
    pass

  def backward(self):
    pass

In [6]:
class Subsampling(Pool):
  def initialise_weights(self, input_shape):
    n = 1 if len(input_shape) == 2 else input_shape[2]
    self.w = cp.random.rand(n)
    self.b = cp.random.rand(n)
    print(input_shape)
    return super().initialise_weights(input_shape)

  def pooling_function(self, matrix):
    return cp.sum(matrix)

  def forward(self, x):
    out = super().forward(x)

    if len(out.shape) == 2:
      return out * self.w[0] + self.b[0]
    
    for i in range(out.shape[2]):
      out[:,:,i] = out[:,:,i] * self.w[i] + self.b[i]
    
    return out

In [7]:
class MaxPool(Pool):
  def __init__(self, k):
    super().__init__(k)

  def pooling_function(self, matrix):
    return cp.max(matrix)

In [8]:
# cp.zeros((5,5,6))

In [9]:
class Flatten(Layer):
  def initialise_weights(self, input_shape):
    out_shape = 1
    for i in input_shape:
      out_shape *= i
    return (out_shape,1)

  def forward(self, x):
    return x.reshape((-1,1))

  def backward(self):
    pass

In [10]:
class Fully_Connected(Layer):
  def __init__(self, units):
    self.units = units
    self.weights = []

  def initialise_weights(self, input_shape):
    self.weights = cp.random.rand(self.units, input_shape[0] + 1)
    return (self.units,1)

  def forward(self, x):
    x = x.reshape((-1,1))
    return cp.dot(weights, cp.vstack((x,1)))

  def backward(self):
    pass

In [11]:
class RBF(Layer):
  def __init__(self, units):
    self.units = units
    self.weights = []

  def initialise_weights(self, input_shape):
    self.weights = cp.random.rand(self.units, input_shape[0] + 1)
    return (self.units,1)

  def forward(self, x):
    x = x.reshape((-1,1))
    out = self.weights - x.T
    out = cp.square(out)
    out = np.sum(out, axis = 1)
    return out

  def backward(self):
    pass

In [12]:
# a = MaxPool(3)

In [13]:
# a.forward(cp.array(cp.zeros((4,4,2))))

In [14]:
# f = Flatten()

In [15]:
# f.forward(cp.array(cp.zeros((4,4,2))))

In [16]:
class Model:
  def __init__(self):
    self.layers = []

  def add(self, layer):
    self.layers.append(layer)

  def compile(self, x_shape, y_shape, print_shapes = False):
    input_shape = x_shape
    for layer in self.layers:
      if print_shapes:
        print(input_shape, layer)
      input_shape = layer.initialise_weights(input_shape)

    if print_shapes:
      print("Output", input_shape)
      
    if input_shape == y_shape:
      print("The layers fit correctly")
    else:
      print("The layers don't fit correctly")
     

  def fit(self, x_train, y_train, epochs, validation_data):
    for epoch in range(epochs):
      layer_in = x_train
      outputs = [layer]

      for layer in self.layers:
        layer_in = layer.forward(layer_in)
    
    print("Model Trained")

  def test(self, x_test):
    outputs = []

    for x in x_test:
      layer_in = x
      for layer in self.layers:
        layer_in = layer.forward(layer_in)
      outputs.append(layer_in)

    return outputs

In [17]:
# cp.array([[1,2], [3,4]]) * cp.array([[[1,2], [3,4]], [[1,3], [3,4]]])

In [18]:
lenet = Model()
lenet.add(Conv((5,5), 6))
lenet.add(Subsampling(2))
lenet.add(Conv((5,5,6), 16))
lenet.add(Subsampling(2))
lenet.add(Conv((5,5, 16), 120))
lenet.add(Flatten())
lenet.add(Fully_Connected(84))
lenet.add(Fully_Connected(10))

In [19]:
lenet.compile((32,32), (10,1), True)

(32, 32) <__main__.Conv object at 0x7f258e07f250>
(28, 28, 6) <__main__.Subsampling object at 0x7f2593b9cad0>
(28, 28, 6)
(14, 14, 6) <__main__.Conv object at 0x7f259418f250>
(10, 10, 16) <__main__.Subsampling object at 0x7f258e055210>
(10, 10, 16)
(5, 5, 16) <__main__.Conv object at 0x7f258e055850>
(1, 1, 120) <__main__.Flatten object at 0x7f258e055fd0>
(120, 1) <__main__.Fully_Connected object at 0x7f258e055890>
(84, 1) <__main__.Fully_Connected object at 0x7f258e055590>
Output (10, 1)
The layers fit correctly


In [20]:
pip install python-mnist

In [21]:
from mnist import MNIST

mndata = MNIST('samples')

images, labels = mndata.load_training()

FileNotFoundError: ignored